# Pandora: a new stereo matching framework
![logo-cnes-triangulaire.jpg](attachment:logo-cnes-triangulaire.jpg)
*Cournet, M., Sarrazin, E., Dumas, L., Michel, J., Guinet, J., Youssefi, D., Defonte, V., Fardet, Q., 2020. Ground-truth generation and disparity estimation for optical satellite imagery. ISPRS - International Archives of the Photogrammetry, Remote Sensing and Spatial Information Sciences.*

# Analysis demo

Imports and external functions

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from pathlib import Path
from IPython.display import Image
import copy
import xarray as xr

In [ ]:
import bokeh.plotting as bpl
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import ColorBar, BasicTicker, LinearColorMapper, Legend
from ipywidgets import interact, Layout
from bokeh.palettes import RdYlBu
from bokeh.io import push_notebook, show, output_notebook
import ipyvolume as ipv
from ipyvolume import widgets

Imports of custom functions for visualization

In [ ]:
from snippets.utils import *

Imports of pandora 

In [ ]:
# Load pandora imports
import pandora
from pandora.img_tools import read_img
from pandora.check_json import check_pipeline_section, concat_conf
from pandora.state_machine import PandoraMachine
from pandora import import_plugin, check_conf

(Optional) If Pandora plugins are to be used, import them

Available Pandora Plugins include :
- MC-CNN Matching cost computation
- SGM Optimization

In [ ]:
# Load plugins
import_plugin()

Provide output directory to write results

In [ ]:
output_dir = os.path.join(os.getcwd(),"output")
# If necessary, create output dir
Path(output_dir).mkdir(exist_ok=True,parents=True)

Provide input data

In [ ]:
# Paths to left and right images
img_left_path = "data/Cones_LEFT.tif"
img_right_path = "data/Cones_RIGHT.tif"
# Paths to masks (None if not provided)
left_mask_path = None
right_mask_path = None

Provide no data configuration

In [ ]:
image_cfg = {'image': {'no_data': 1, 'nodata1': np.nan, 'nodata2': np.nan, 'valid_pixels': 0}}

Read input data and convert to dataset

In [ ]:
img_left = read_img(img_left_path, no_data=image_cfg['image']['nodata1'],
                       mask=left_mask_path)
img_right = read_img(img_right_path, no_data=image_cfg['image']['nodata2'],
                       mask=right_mask_path)

#### Visualize input data

In [ ]:
show_input_images(img_left, img_right)

Load ground truth if available

In [ ]:
# Read image of ground_thruth
ground_truth = read_img("data/Cones_LEFT_GT.tif", np.inf, mask="data/Occlusion_LEFT.png")

# Convert disparity map to Pandora's convention
ground_truth["disparity_map"] = xr.DataArray(np.copy(ground_truth.im),dims=['row', 'col'])
ground_truth["disparity_map"].values = - ground_truth["disparity_map"].values
ground_truth["validity_mask"] = xr.DataArray(np.copy(ground_truth.msk),dims=['row', 'col'])
ground_truth["validity_mask"].values = np.zeros(ground_truth["msk"].values.shape, dtype = int)
inv_idx = np.where(ground_truth["msk"].values == 0)
ground_truth["validity_mask"].values[inv_idx] = pandora.constants.PANDORA_MSK_PIXEL_OCCLUSION


#### Visualize ground truth

The different types of masks can be selected for visualization

In [ ]:
plot_disparity(ground_truth)

#### Run Pandora

Define 2 pipelines

In [ ]:
user_pipeline_cfg_census_sgm = {
                     'pipeline': { 
                         'right_disp_map':{'method': 'accurate'},
                         'matching_cost': {'matching_cost_method': 'census', 'window_size': 5, 'subpix': 1},
                         'optimization' : {'optimization_method': 'sgm'},
                         'disparity': {'disparity_method':'wta', "invalid_disparity": "NaN"},
                         'filter': {'filter_method': 'median'},
                         'refinement': {'refinement_method': 'vfit'},
                         'validation': {'validation_method': 'cross_checking'},
                     }
                     }

In [ ]:
user_pipeline_cfg_zncc = {
                     'pipeline': { 
                         'right_disp_map':{'method': 'accurate'},
                         'matching_cost': {'matching_cost_method': 'zncc', 'window_size': 5, 'subpix': 1},
                         'disparity': {'disparity_method':'wta', "invalid_disparity": "NaN"},
                         'refinement': {'refinement_method': 'vfit'},
                         'validation': {'validation_method': 'cross_checking'},
                     }
                     }

Instantiate and run the machine with the first configuration

In [ ]:
pandora_machine = PandoraMachine()

In [ ]:
cfg_census_sgm = check_pipeline_section(user_pipeline_cfg_census_sgm, pandora_machine)['pipeline']

In [ ]:
disp_min = -60
disp_max = 0

In [ ]:
pandora_machine.run_prepare(cfg_census_sgm, img_left, img_right, disp_min, disp_max)

In [ ]:
pandora_machine.run('matching_cost', cfg_census_sgm)
left_cv_census = copy.deepcopy(pandora_machine.left_cv)
right_cv_census = copy.deepcopy(pandora_machine.right_cv)
    
pandora_machine.run('optimization', cfg_census_sgm)
left_cv_census_sgm = copy.deepcopy(pandora_machine.left_cv)
right_cv_census_sgm = copy.deepcopy(pandora_machine.right_cv)
    
pandora_machine.run('disparity', cfg_census_sgm)
left_disparity_map_census_sgm = copy.deepcopy(pandora_machine.left_disparity)
right_disparity_map_census_sgm = copy.deepcopy(pandora_machine.right_disparity)

pandora_machine.run('refinement', cfg_census_sgm)
left_disp_map_refined_census_sgm = copy.deepcopy(pandora_machine.left_disparity)
right_disp_map_refined_census_sgm = copy.deepcopy(pandora_machine.right_disparity)
    
pandora_machine.run('filter', cfg_census_sgm)
left_disp_map_filtered_census_sgm = copy.deepcopy(pandora_machine.left_disparity)
right_disp_map_filtered_census_sgm = copy.deepcopy(pandora_machine.right_disparity)

pandora_machine.run('validation', cfg_census_sgm)
left_disp_map_validated_census_sgm = copy.deepcopy(pandora_machine.left_disparity)
right_disp_map_validated_census_sgm = copy.deepcopy(pandora_machine.right_disparity)

Instantiate and run the machine with the second configuration

In [ ]:
pandora_machine = PandoraMachine()

In [ ]:
cfg_zncc = check_pipeline_section(user_pipeline_cfg_zncc, pandora_machine)['pipeline']

In [ ]:
disp_min = -60
disp_max = 0

In [ ]:
pandora_machine.run_prepare(cfg_zncc, img_left, img_right, disp_min, disp_max)

In [ ]:
pandora_machine.run('matching_cost', cfg_zncc)
left_cv_zncc = copy.deepcopy(pandora_machine.left_cv)
right_cv_zncc = copy.deepcopy(pandora_machine.right_cv)
  
pandora_machine.run('disparity', cfg_zncc)
left_disparity_map_zncc = copy.deepcopy(pandora_machine.left_disparity)
right_disparity_map_zncc = copy.deepcopy(pandora_machine.right_disparity)

pandora_machine.run('refinement', cfg_zncc)
left_disp_map_refined_zncc = copy.deepcopy(pandora_machine.left_disparity)
right_disp_map_refined_zncc = copy.deepcopy(pandora_machine.right_disparity)

pandora_machine.run('validation', cfg_zncc)
left_disp_map_validated_zncc = copy.deepcopy(pandora_machine.left_disparity)
right_disp_map_validated_zncc = copy.deepcopy(pandora_machine.right_disparity)

## Analyze results

#### Visualize output disparity for both pipelines

In [ ]:
plot_disparity(left_disp_map_validated_census_sgm)

In [ ]:
plot_disparity(left_disp_map_validated_zncc)

#### Visualize and compare the different cost volumes

Plot 1 cost volume

In [ ]:
plot_1_cost_volume(left_cv_census_sgm, left_disparity_map_census_sgm, "Cost volume with Census matching cost and SGM optimization step")

Plot 2 cost volumes

In [ ]:
cv_zncc = get_3D_cost_volume(left_cv_zncc, left_disparity_map_zncc)
cv_census_sgm = get_3D_cost_volume(left_cv_census_sgm, left_disparity_map_census_sgm)

In [ ]:
out = widgets.Output()
widgets.HBox([cv_zncc, cv_census_sgm], layout=Layout(width='880px', height='550px'))

#### Compare the disparity maps after the refinement step of the two pipelines

In [ ]:
compare_2_disparities(left_disp_map_refined_census_sgm, "Disparity map Census SGM", left_disp_map_refined_zncc, "Disparity map Zncc")

#### Compare the output disparity map of the two pipelines

The invalid masks can be hidden if the legend is clicked

In [ ]:
compare_2_disparities( left_disp_map_validated_zncc, "Disparity map ZNCC", left_disp_map_validated_census_sgm, "Disparity map Census SGM")

#### Compare the output disparity map with the ground truth for the zncc pipeline

In [ ]:
compare_2_disparities(ground_truth, "Ground truth", left_disp_map_validated_zncc, "Disparity map ZNCC")

#### Plot error and statistics with variable error threshold  for the zncc pipeline

In [ ]:
error_plot(left_disp_map_validated_zncc, ground_truth, "Error Zncc")

#### Compare the disparity map with the error at a given threshold for both pipelines

In [ ]:
error_census_sgm = get_error(left_disp_map_validated_census_sgm, ground_truth, threshold = 1)

In [ ]:
compare_disparity_and_error(left_disp_map_validated_census_sgm, "Disparity map Census SGM", error_census_sgm, "Error with threshold 1",)

In [ ]:
error_zncc = get_error(left_disp_map_validated_zncc, ground_truth, threshold = 1)

In [ ]:
compare_disparity_and_error(left_disp_map_validated_zncc, "Disparity map Zncc", error_zncc, "Error with threshold 1",)

#### See the percentage of improvement or degradation on the error of Census-SGM in respect to ZNCC

Negative percentages (blue points) mean an error reduction in respect to the reference error.


In [ ]:
compare_2_error_maps(reference_error=error_zncc, second_error=error_census_sgm)

#### Compare disparity maps with the ground truth and error for the ZNCC pipeline

In [ ]:
compare_3_disparities_and_error(left_disparity_map_zncc, "Disparity map Zncc after disparity step", left_disp_map_validated_zncc, "Disparity map Zncc with refinement and cross validation", ground_truth, "Ground truth", error_zncc, "Error with threshold 1",)